In [9]:
from elftools.elf.elffile import ELFFile
import frida
import sys
from time import sleep

In [10]:
def generate_function_list(binary):
    """
    Generate a list of all the functions defined by the target executable.
    """
    functions = []
    with open(binary, "rb") as f:
        elf = ELFFile(f)
        for section in elf.iter_sections():
            if section.header.sh_type == 'SHT_SYMTAB':
                for symbol in section.iter_symbols():
                    if symbol.entry.st_info.type == 'STT_FUNC':
                        functions.append(symbol.name)
    return functions

In [11]:
binary_path = "./test/test"
args = ["arg1", "arg2"]

function_list = generate_function_list(binary_path)
print(function_list)

['deregister_tm_clones', 'register_tm_clones', '__do_global_dtors_aux', 'frame_dummy', '__libc_start_main@GLIBC_2.34', 'puts@GLIBC_2.2.5', 'h', '_fini', 'f', 'printf@GLIBC_2.2.5', '_start', 'g', 'main', '__cxa_finalize@GLIBC_2.2.5', '_init']


In [15]:
entries = []

def on_message(message, data):
    print(message)
    if message["type"] == "send" and message["payload"] != "done":
        function_name = message["payload"]["function"]
        try:
            function_args = message["payload"]["args"]
            io="input"
        except:
            function_args=message["payload"]["ret"]
            io="output"
        entries.append((function_name,function_args))

# Run the binary
process = frida.spawn(binary_path, argv=[binary_path] + args)

sleep(1)

session = frida.attach(process)
for f in ['f','g','h']:
    script = session.create_script("""
            console.log('Called function '+DebugSymbol.getFunctionByName('"""+f+"""'))
            Interceptor.attach(DebugSymbol.getFunctionByName('"""+f+"""'), {
                onEnter: function (args) {
                    send({function: '"""+f+"""', args: args[0]})
                    console.log('onEnter function '+DebugSymbol.getFunctionByName('"""+f+"""'))
                },
                onLeave: function (retval) {
                	send({function: '"""+f+"""', ret: retval})
                    console.log('onLeave function '+DebugSymbol.getFunctionByName('"""+f+"""'))

                }
            });
        """) 
    
    script.on("message", on_message)
    script.load()

frida.resume(process)

Called function 0x55e5dc25e1a9
Called function 0x55e5dc25e18a
Called function 0x55e5dc25e169
{'type': 'send', 'payload': {'function': 'f', 'args': '0x3'}}
onEnter function 0x55e5dc25e1a9
{'type': 'send', 'payload': {'function': 'h', 'args': '0x6'}}
Hello
0x55e5dc25e169
onEnter function 0x55e5dc25e169
{'type': 'send', 'payload': {'function': 'h', 'ret': '0x6'}}
onLeave function 0x55e5dc25e169
{'type': 'send', 'payload': {'function': 'f', 'ret': '0xf'}}
onLeave function 0x55e5dc25e1a9


In [4]:
print( list(set([3,2,4])&set([1,2,3])))

[2, 3]
